# Accessing SDSS-V Spectra (BOSS)
This notebook demonstrates methods that can be used for programmatic access to SDSS-V data products from the BOSS spectrograph. Prior to running this notebook, you should have already done the following:

* If not yet an official SDSS-V collaboration member:
  * Sign up for an SDSS-V Twiki Account (follow [instructions here](https://wiki.sdss.org/))
  * Follow the [SDSS-V Welcome instructions](https://sdss-wiki.atlassian.net/wiki/spaces/SDSS/pages/13343105/Welcome+to+SDSS-V) for new collaborators
* Get the SDSS-V data access credentials so you can access [https://data.sdss5.org/sas/sdsswork/](https://data.sdss5.org/sas/sdsswork/)
* [Create a .netrc file](https://sdss-access.readthedocs.io/en/latest/auth.html) in your home directory with SDSS-V username and password

In [ ]:
import os
import io
import sys
import time
import requests
import numpy as np
import pandas as pd
import astropy.units as u
import matplotlib.pyplot as plt

from netrc import netrc
from astropy.io import fits
from astropy.coordinates import SkyCoord
from astropy.coordinates import match_coordinates_sky
from astroquery.vizier import Vizier
from IPython.display import display, Markdown

# SDSS-V Tools
sys.path.insert(1, '../')
import sdssv_utils as sdssv

# BOSS Spectra
The [BOSS instruments](https://www.sdss.org/instruments/boss-spectrographs/) are optical spectrographs with wavelength coverage of 3600-10400 Angstrom and resolutions of R~2000. There are two identical BOSS spectrographs, one located in the northern hemisphere at Apache Point Observatory (APO), and one in the southern hemisphere at Las Campanas Observatory (LCO), both on 2.5-m telescopes. Each spectrograph has 500, 2-arcsec fibers with robotic fiber positioners.

## Pipeline Versions and [Internal Product Launches (IPLs)](https://wiki.sdss.org/display/IPL/Internal+Product+Launch)
Whenever you access SDSS data, you will have to choose from which pipeline version or IPL you wish to get data from. IPLs contain data that have undergone more rigorous data quality and assurance processes, but only provide data up to a fixed cutoff date and for specific observatories for each instrument (e.g for IPL-3, BOSS spectra are only provided from the northern APO site up to MJD = 60130). Similarly, specific pipeline versions, like "v6_1_3", are also often limited in scope, since they are usually developed to support specific IPLs or future public data releases. When a new pipeline version is released, all old data is reprocessed. Older pipelines are no longer run on newer data, but the data data products produced by older pipelines remain available. On the other hand, you can get more recent spectra from all observing sites by downloading directly from the ''master'' directory (e.g. https://data.sdss5.org/sas/sdsswork/bhm/boss/spectro/redux/master), but you are more likely to encounter poor-quality data or other bugs/issues. 

In addition, the data products being provided will change from version to version and for each IPL. For example, the BOSS pipeline used for IPLs 1 and 2 (v6.0.9) only performed coadds for spectra taken on the same MJD, but starting with [IPL-3](https://wiki.sdss.org/display/IPL/BOSS+Field+Epoch+Coadds+and+Custom+Coadd) and in pipeline versions v6_1_1 and later, multi-day and custom coadds requested by different working groups will also be provided, which can be accessed using the "epoch" and "allepoch" COADD_TYPE options. Many other types of data products (e.g. stellar parameters) are also provided that are not shown in this notebook.

So in short, it's important to consider which data products or pipeline version you'd like to use for a given project. This notebook and the `sdssv_utils.py` script are currently capable of providing access to SDSS-V BOSS spectra from IPLs 1, 2 and 3, from pipeline versions 6.0.9, 6.1.0, 6.1.1, 6.1.2, 6.1.3, and from the master directory. 

## Set Some Global Variables
Supported Options:
* `VERSION`:  <span style="font-family:Courier New">ipl-1, ipl-2, ipl-3, v6_0_9, v6_1_0, v6_1_1, v6_1_2, v6_1_3, master</span>
* `SPALL_TYPE`:  <span style="font-family:Courier New">lite, full</span>
* `SPEC_TYPE`:  <span style="font-family:Courier New">lite, full</span>
* `COADD_TYPE`:  <span style="font-family:Courier New">daily, epoch, allepoch</span>

<b>NOTE: "epoch" and "allepoch" COADD_TYPE options only work with IPL-3, v6_1_1, v6_1_2, and v6_1_3!</b>

In [ ]:
VERSION = "master"                # BOSS Pipeline/Release Version
SPALL_TYPE = 'lite'               # Type of spAll file being used. Either full or lite.
SPEC_TYPE = 'full'                # Either full or lite. lite is coadd only. full is coadd + individual spectra
COADD_TYPE = 'daily'              # Determines what type of spectral coadd to download
SPALL_DIR = f"./spall/{VERSION}/{COADD_TYPE}"  # Local directory where spAll file exists

## Set Some Base Remote URLs using Choices Above

In [ ]:
# Set sdss-access RELEASE name
if 'ipl' in VERSION:
    RELEASE = VERSION.replace("-","").upper()
else:
    RELEASE = 'sdsswork'

# Set SAS Base and Remote URLs
if 'ipl' in VERSION:
    PIPELINE_VERSION = sdssv.get_IPL_BOSS_pipeline_version(VERSION)        # Pipeline version used for given IPL
    SAS_BASE = f'https://data.sdss5.org/sas/{VERSION}'                # Base URL for the SDSS Science Archive Server
    REMOTE_DIR = f"{SAS_BASE}/spectro/boss/redux/{PIPELINE_VERSION}"  # URL where spAll file and spectra can be found for BOSS
else:
    PIPELINE_VERSION = VERSION
    SAS_BASE = f'https://data.sdss5.org/sas/sdsswork'
    REMOTE_DIR = f"{SAS_BASE}/bhm/boss/spectro/redux/{VERSION}" 

print(f'Remote directory: {REMOTE_DIR}')

## Check for the SpAll File
The spAll file contains a list of reduced BOSS spectra available from SDSS-V. Each pipeline version or IPL has an associated spAll file, which can be found in the version/IPL's subdirectory here:
https://data.sdss5.org/sas/sdsswork/bhm/boss/spectro/redux/

Again, several pipeline versions and IPLs exist, so make sure you are downloading the correct file for the version you wish to use. In this notebook, we'll be checking for the **LITE** version of the **master** spAll file. There is also a FULL spAll version with more parameters, but it is a much larger file. At present (2024 May 21), the LITE version of the master spAll file is about 1.5 Gb compressed and 6.5 Gb uncompressed.

In [ ]:
# Make sure SPALL_DIR exists
if not os.path.exists(SPALL_DIR):
    os.makedirs(SPALL_DIR)

# Define spAll filename and path
spall_filename = sdssv.get_spall_filename(VERSION, SPALL_TYPE, COADD_TYPE)
spall_filepath = f"{SPALL_DIR}/{spall_filename}"

sdssv.download_spall(
    VERSION,
    coadd_type=COADD_TYPE,
    spall_type=SPALL_TYPE,
    save_dir=SPALL_DIR,
    netfile=None,   # Defaults to looking in home directory for .netrc file
    overwrite=False # If TRUE, will download spall file even if local file already exists
)                   # If FALSE, will only download spall file if it doesn't already exist

## Load in the SpAll File and Show Sky Coverage

In [ ]:
# Load spAll
with fits.open(spall_filepath) as hdu:
    spall = hdu[1].data

# Check for any NaN coordinates
nan_coords = ((np.isnan(spall.FIBER_RA)) | 
              (np.isnan(spall.FIBER_DEC)))
numnan = sum(nan_coords)
spall = spall[~nan_coords]

# Get some metadata
Nspall = len(spall)
mjd_min = np.min(spall.MJD)
mjd_max = np.max(spall.MJD)
try:
    apo_nspec = sum(spall.OBS == 'APO')
    lco_nspec = sum(spall.OBS == 'LCO')
except: # For earlier versions when only APO data exists
    apo_nspec = Nspall
    lco_nspec = 0
display(Markdown("**Summary of spAll File:**"))
print(
    f"\n{Nspall} spectra available in SDSS-V {VERSION.upper()} spAll-{SPALL_TYPE} file with {COADD_TYPE.upper()} coadds." +
    f"\n   MJD Min = {mjd_min:.0f}\n   MJD Max = {mjd_max:.0f}" +
    f"\n  APO Spec = {apo_nspec}\n  LCO Spec = {lco_nspec}" +
    f"\nNaN Coords = {numnan}\n"
)

# Get a small sample and display one entry of the spAll file
display(Markdown("<br />**Sample spAll Entry:**"))
spall_sample = pd.DataFrame(
    spall[
        (spall.SN_MEDIAN_ALL>20) &
        (spall.GAIA_G_MAG>10) &
        (spall.NEXP <= 6)
    ][0:10].tolist(), 
    columns=spall.columns.names
)
display(Markdown(spall_sample.head(1).to_markdown()))

# Create spAll coordinates object
spall_ra = spall.FIBER_RA
spall_dec = spall.FIBER_DEC
spall_coord = SkyCoord(
    ra=spall_ra*u.deg,
    dec=spall_dec*u.deg,
    frame='icrs'
)

# Display Sky Positions of random selection of SDSS-V Spectra
display(Markdown("<br />**Sky Coverage:**"))
Nplot = 1000000 # Max Number of random sources to plot. Nplot = None will plot ALL sources
if apo_nspec>0 and lco_nspec>0:
    ax = sdssv.sky_plot(spall_coord[spall.OBS=='APO'], size=Nplot, color='coral',label='APO');
    ax = sdssv.sky_plot(spall_coord[spall.OBS=='LCO'], size=Nplot, ax=ax, color='skyblue',label='LCO');
else:
    ax = sdssv.sky_plot(spall_coord, size=Nplot, label='APO');
ax.legend(loc='upper right', fontsize=14, markerscale=5, handletextpad=0.5);

## Try Cross Matching a Single Object's Coordinates with SDSS-V Objects
This cell will find all rows within the spAll file that match within a defined search radius of an object's coordinates. 

In [ ]:
# Coordinates for a random star from spAll
obj_ra = spall_sample.FIBER_RA.iloc[0]
obj_dec = spall_sample.FIBER_DEC.iloc[0]

# Perform cross match
radius = 2.0 # arcsec
idx_spall = sdssv.singleObject_cross_match(
    obj_ra, obj_dec, spall_coord, radius=radius
)
spall_match = pd.DataFrame(
    spall[idx_spall].tolist(), 
    columns=spall.columns.names
)
spall_match.drop_duplicates(
    subset=['MJD_FINAL','SPEC_FILE'], inplace=True, ignore_index=True
)
Nmatch = len(spall_match)
print(f'Found {Nmatch} SDSS-V spectrum file(s) within {radius:.1f} arcsec radius\n')

# Generate full URL and print out info
for i,row in spall_match.iterrows():
    
    catalogid = row.CATALOGID
    field = row.FIELD
    mjd = row.MJD
    specfile = row.SPEC_FILE
    
    if COADD_TYPE == 'epoch':
        spec_url = f"{REMOTE_DIR}/epoch/spectra/{SPEC_TYPE}/{field:06d}/{mjd:5d}/{specfile}"
    elif COADD_TYPE == 'allepoch':
        spec_url = f"{REMOTE_DIR}/spectra/{SPEC_TYPE}/allepoch/{mjd:5d}/{specfile}"
    else:
        spec_url = f"{REMOTE_DIR}/spectra/{SPEC_TYPE}/{field:06d}/{mjd:5d}/{specfile}"

    print(f'SPEC-{i+1}\n----------------------')
    print(' FIBER-RA = {:10.6f}'.format(row.FIBER_RA))
    print('FIBER-DEC = {:10.6f}'.format(row.FIBER_DEC))
    print('CATALOGID = {:d}'.format(row.CATALOGID))
    print('    FIELD = {:d}'.format(row.FIELD))
    print('      MJD = {:d}'.format(row.MJD))
    print('    CLASS = {}'.format(row.CLASS))
    print(' SUBCLASS = {}'.format(row.SUBCLASS))
    print('   CARTON = {}'.format(row.FIRSTCARTON))
    print('SPEC FILE = {}'.format(row.SPEC_FILE))
    print(f' SPEC URL = {spec_url}\n')
    
display(Markdown(spall_match.to_markdown()))

### Download Spectrum to Directory of Choice using urllib
This cell will use the SDSS-V username and password stored in the .netrc file in your home directory. If you don't have a .netrc file in your home directory yet, or it doesn't yet contain an entry for SDSS-V, you can create one following the instructions [here](https://sdss-access.readthedocs.io/en/latest/auth.html). The urllib download function contained in the sdssv_utils.py script (download_spec_urllib) uses multiprocessing, which sometimes does not play nice with different operatign systems. Let me know (zvanderb@caltech.edu) if you experience any problems.

In [ ]:
SAVE_DIR = f"./spec_files/{VERSION}/{COADD_TYPE}"
if not os.path.exists(SAVE_DIR):
    os.makedirs(SAVE_DIR)

# Download the spectra
spec_paths_urllib = sdssv.download_spec_urllib(
    spall_match,
    VERSION,
    save_dir=SAVE_DIR,
    coadd_type=COADD_TYPE,
    spectype=SPEC_TYPE,
    netfile=None,   # Defaults to looking in home directory for .netrc file
    threads=8,      # Number of parallel download streams
    chunksize=100,  # Number of jobs to submit at once
    sleeptime=3.0,  # Time (sec) to pause between job submissions
    overwrite=False # Whether to overwrite existing local spectra
)

# Print local paths to downloaded files
print('\nLocal File Paths to Spectra:')
for sp in spec_paths_urllib:
    print(sp)

### Load and Plot the Spectrum (Coadd Only)

In [ ]:
spec_idx = 0
specname = spec_paths_urllib[spec_idx]
spec_data = sdssv.load_SDSS_spectrum(
    specname, 
    method='COADD' # Only the co-added spectrum will be returned
)
sdssv.plot_spectrum(
    spec_data['COADD'], 
    spall_dat=spall_match.iloc[spec_idx],
    show_sky=False,
    show_uncertainty=True
);
plt.close()

### Load and Plot Individual Spectra
If you are downloading the **full** versions of the spectra, there may be multiple individual exposures comprising the COADD that you can extract from the spec files. If using the **lite** versions of the spectra, only coadds are available within the files.

In [ ]:
specname = spec_paths_urllib[spec_idx]
spec_data = sdssv.load_SDSS_spectrum(
    specname, 
    method='ALL' # All individual spectra + coadd are returned
)

if SPEC_TYPE == 'lite':
    print('Only COADDs are available in LITE spec files.')
else:    
    for key in spec_data.keys():
        
        if key == 'COADD':
            continue
            
        sdssv.plot_spectrum(
            spec_data[key],
            show_uncertainty=True
        );
        plt.close()

## Downloading Spectra for Multiple Objects
As an example of cross matching multiple objects with the spAll file, the following cells will look for all spectra obtained for objects within a specific SDSS-V carton, in this case the **mwm_wd_core** carton. [SDSS-V cartons](https://www.sdss.org/dr18/targeting/) are groups of objects with specific survey requirements (e.g. cadence, instrument, and dark vs. bright time) and well-defined selection criteria. All SDSS-V carton targeting information was released as part of SDSS DR18 (see [Almeida et al. 2023](https://ui.adsabs.harvard.edu/abs/2023ApJS..267...44A/abstract)) and is publicly available information.

This is a slightly roundabout way to get spectra for a specific carton since the spAll files themselves contain a **FIRSTCARTON** column that can used to identify objects within a specific carton, but this is just to help illustrate multi-object cross-matching as well as demonstrate the functions in *sdssv_utils* that provide carton information.

First let's print out all of the available SDSS-V carton names and IDs:

In [ ]:
sdssv.print_SDSSV_cartons(cartons='all')  # Change cartons to 'mwm' or 'bhm' to only show those groups

## Now let's get metadata for sources in the mwm_wd_core (**ID = 585**) carton and cross match with the spAll file

In [ ]:
# Query carton metadata
carton_result = sdssv.sdssv_carton_query(585, qlimit=500000)

# Filter out faint sources for better visualization
maglim = 19.0
carton_result = carton_result.loc[carton_result.gaia_g < maglim].reset_index(drop=True)

# Get source coordinates
sample_ra = carton_result.ra.values
sample_dec = carton_result.dec.values

# Perform cross match
radius = 1.0 # arcsec
idx_sample, idx_spall = sdssv.multiObject_cross_match(
    sample_ra, 
    sample_dec,
    spall_coord,
    radius=radius
)

# Create dataframes with objects matched to spectra
spall_sample_match = pd.DataFrame(
    spall[idx_spall].tolist(),
    columns=spall.columns.names
)

# Ensure all zero-valued G-band magnitudes are replaced
spall_sample_match['GAIA_G_MAG'] = carton_result['gaia_g'][idx_sample].values

# Print out crossmatch results
Nsample_unique = len(spall_sample_match.CATALOGID.unique())
print(f"\n{Nsample_unique} Unique Sources with G < {maglim:.0f} mag Matched to {len(spall_sample_match)} SDSS-V Spec Files\n")

### Print Out Total Number of Spectra per Matched Source

In [ ]:
for i,src in enumerate(spall_sample_match.CATALOGID.unique()):

    if i > 9: break
        
    df_entries = spall_sample_match.loc[spall_sample_match.CATALOGID ==src]
    Nspecfiles = len(df_entries)
    Nspectotal = df_entries.NEXP.sum()
    print(f"ID = {src:17d}:   {Nspectotal:3d} spectra total across {Nspecfiles:2d} file(s)")

### Download Spectra using urllib
Only get spectra for first 10 sources

In [ ]:
if not os.path.exists(SAVE_DIR):
    os.mkdir(SAVE_DIR)

# Download the spectra
spec_paths_urllib_multiobject = sdssv.download_spec_urllib(
    spall_sample_match.iloc[:10],
    VERSION,
    coadd_type=COADD_TYPE,
    save_dir=SAVE_DIR,
    spectype=SPEC_TYPE,
    netfile=None,   # Defaults to looking in home directory for .netrc file
    threads=8,      # Number of parallel download streams
    chunksize=100,  # Number of jobs to submit at once
    sleeptime=3.0,  # Time (sec) to pause between job submissions
    overwrite=False # Whether to overwrite existing local spectra
)

# Print local paths to downloaded files
print('\nLocal File Paths to Spectra:')
for sp in spec_paths_urllib_multiobject:
    print(sp)

### Plot Each Coadd Spectrum

In [ ]:
for spec_idx,specname in enumerate(spec_paths_urllib_multiobject):
    
    spec_data = sdssv.load_SDSS_spectrum(
        specname, 
        method='COADD' # Only the co-added spectrum will be returned
    )
    sdssv.plot_spectrum(
        spec_data['COADD'], 
        spall_dat=spall_sample_match.iloc[spec_idx],
        show_sky=False,
        show_uncertainty=True
    )
    plt.close()

### Look at All Individual Spectra For a Source Across Multiple Files

In [ ]:
# Get paths to files for multi-epoch spectra
obj_idx = 0
catID = spall_sample_match.CATALOGID.unique()[obj_idx]
catID_idx = spall_sample_match.CATALOGID == catID
catID_entries = spall_sample_match.loc[catID_idx].reset_index(drop=True)
catID_specfiles = [x for idx,x in zip(catID_idx,spec_paths_urllib_multiobject) if idx]
catID_nspec = catID_entries.NEXP.sum()

if SPEC_TYPE == 'lite':
    print('Only COADDs are available in LITE spec files.')
else:
    print(f'\nPlotting {catID_nspec} individual spectra for Catalog ID {catID}\n')
    
    # Load and plot each spectrum
    for i,row in catID_entries.iterrows():
    
        specname = catID_specfiles[i]
        spec_data = sdssv.load_SDSS_spectrum(
            specname, 
            method='ALL' # All individual spectra + coadd are returned
        )
        
        for key in spec_data.keys():
            if key == 'COADD':
                continue
                
            sdssv.plot_spectrum(
                spec_data[key],
                spall_dat = catID_entries.iloc[i],
                show_uncertainty=True
            );
            plt.close()
